In [1]:
import numpy as np
import pandas as pd
import utilities
import statistics
import matplotlib.pyplot as plt
import torch

/home/users/lewinsda/.conda/envs/daniel_thesis_2/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
spg_data = pd.read_table("/home/groups/ConradLab/daniel/GNN_cell_annotation/data/jung/GSM3102982_Pouf5SPGmat.txt", index_col = 0)
spc1_data = pd.read_table("/home/groups/ConradLab/daniel/GNN_cell_annotation/data/jung/GSM3102983_SPCImat.txt", index_col = 0)
spc2_data = pd.read_table("/home/groups/ConradLab/daniel/GNN_cell_annotation/data/jung/GSM3102984_SPCIImat.txt", index_col = 0)
spd_data = pd.read_table("/home/groups/ConradLab/daniel/GNN_cell_annotation/data/jung/GSM3102985_SPDmat.txt", index_col = 0)

In [3]:
spc1_data.head()

,GTTTTCTCGTT,GGGACCACGCC,GTCGTGCATTT,GTTGACTTCCG,AATAACACAGC,GCTTCTTTGCT,GGTTTTCTCGT,ACAGTCAGGGT,TCCCTAATCAG,GTCGTCATTTA,...,TTTCTTCCCAT,CGATGGGTTTT,AACATACAAAG,CCTCTTTCATT,ATCCCTTTCGA,CCCGCGTCGCT,GGGTTCACGCC,GTCACTAGGTC,GTCCCAATGTC,GTTCATTACAC
0610007P14Rik,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0610009B22Rik,1,1,0,2,0,1,0,1,0,1,...,0,0,1,0,0,0,0,0,0,0
0610009L18Rik,4,4,0,1,0,0,0,1,1,0,...,0,1,0,0,0,0,0,0,0,0
0610009O20Rik,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0610010F05Rik,0,2,0,0,1,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
merged_data = spg_data.merge(spc1_data, how='inner', left_index=True, right_index=True)
merged_data = merged_data.merge(spc2_data, how='inner', left_index=True, right_index=True)
merged_data = merged_data.merge(spd_data, how='inner', left_index=True, right_index=True)

In [5]:
merged_data.shape

(6354, 1192)

In [6]:
#merged_data.loc["Dmrt1",:]

In [6]:
merged_data.loc[["0610009L18Rik", "0610012G03Rik"], "TTCGTGTCGAA"]

0610009L18Rik    0
0610012G03Rik    2
Name: TTCGTGTCGAA, dtype: int64

In [7]:
genes = merged_data.index.array
cell_names = merged_data.columns.array

In [8]:
merged_data = np.asmatrix(merged_data)
merged_data = merged_data.transpose()
merged_data.shape

(1192, 6354)

In [9]:
spg_labels = ["spg"] * len(spg_data.columns)
spc1_labels = ["spc"] * len(spc1_data.columns)
spc2_labels = ["spc"] * len(spc2_data.columns)
spd_labels = ["spd"] * len(spd_data.columns)
merged_labels = spg_labels + spc1_labels + spc2_labels + spd_labels
len(merged_labels)

1192

In [10]:
unique, counts = np.unique(merged_labels, return_counts=True)
counts

array([597, 299, 296])

In [11]:
# read in dataset
X, keep_cells = utilities.preprocess(np.array(merged_data), normalize=False, run_pca=False, scale=False)
X.shape

(692, 6354)

In [12]:
merged_labels = np.array(merged_labels)[keep_cells]
cell_names = cell_names[keep_cells]
len(merged_labels)

692

In [13]:
unique, counts = np.unique(merged_labels, return_counts=True)
counts

array([244, 156, 292])

In [14]:
# randomize
np.random.seed(8)
p = np.random.permutation(len(merged_labels))
merged_labels = merged_labels[p]
X = X[p,:]
cell_names = cell_names[p]

In [15]:
# write out
final_counts = pd.DataFrame(X)
final_counts.index = cell_names
final_counts.columns = genes

In [16]:
final_counts.head()

,0610009B22Rik,0610009L18Rik,0610009O20Rik,0610010F05Rik,0610011F06Rik,0610012G03Rik,0610030E20Rik,0610037L13Rik,0610040B10Rik,1110001J03Rik,...,Zzz3,mt-Co1,mt-Cytb,mt-Nd1,mt-Nd2,mt-Nd3,mt-Nd4,mt-Nd5,mt-Rnr1,mt-Rnr2
TTCGTGTCGAA,0,0,0,0,0,2,0,0,0,0,...,0,12,30,27,12,0,15,8,9,32
GTACAATGATG,0,0,0,1,0,2,0,0,1,0,...,1,49,58,39,18,1,18,18,27,88
TCGAGGACGTT,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
GTGAATTTTCT,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,11
CCACACGCGGT,0,0,0,0,0,0,0,0,0,0,...,1,24,29,29,7,0,16,6,11,50


In [17]:
final_counts.loc["TTCGTGTCGAA", ["0610009L18Rik", "0610012G03Rik"]]

0610009L18Rik    0
0610012G03Rik    2
Name: TTCGTGTCGAA, dtype: int64

In [19]:
#final_counts.to_csv("/home/groups/ConradLab/daniel/sharp_data/jung/counts.csv")

In [18]:
all_labels = pd.read_csv("/home/groups/ConradLab/daniel/sharp_data/jung/preds.csv", index_col=0)

In [19]:
_,marker_names = utilities.read_marker_file("/home/groups/ConradLab/daniel/sharp_data/jung/markers.txt")
marker_names

['spg', 'spc', 'spd']

In [20]:
all_labels_factored = utilities.factorize_df(all_labels, marker_names)
encoded_labels = utilities.encode_predictions(all_labels_factored)
encoded_labels

array([[0., 0., 3.],
       [0., 0., 3.],
       [0., 3., 0.],
       ...,
       [1., 0., 0.],
       [3., 0., 0.],
       [0., 3., 0.]])

In [21]:
all_labels_factored

,scina,scsorter,sctype
0,2,2,2
1,2,2,2
2,1,1,1
3,1,1,1
4,2,2,2
...,...,...,...
687,2,2,2
688,2,2,2
689,-1,-1,0
690,0,0,0


In [22]:
merged_labels = pd.factorize(merged_labels, sort=True)[0]
merged_labels

array([2, 2, 1, 1, 2, 2, 2, 2, 2, 1, 0, 0, 2, 2, 2, 0, 2, 2, 1, 2, 1, 1,
       1, 2, 1, 1, 2, 2, 2, 0, 1, 2, 2, 2, 1, 2, 2, 0, 2, 0, 2, 0, 0, 2,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 2, 2, 2, 2, 1, 2, 1, 2, 1, 0,
       2, 1, 2, 0, 1, 0, 2, 0, 1, 0, 1, 1, 0, 2, 0, 2, 2, 0, 1, 0, 0, 2,
       2, 1, 2, 0, 0, 0, 0, 1, 2, 0, 2, 0, 0, 0, 0, 2, 2, 2, 1, 2, 0, 0,
       0, 0, 2, 2, 2, 0, 2, 2, 1, 2, 1, 1, 1, 0, 1, 0, 1, 1, 2, 1, 2, 2,
       2, 0, 2, 1, 2, 1, 0, 0, 2, 2, 2, 2, 0, 1, 2, 2, 1, 2, 0, 0, 0, 2,
       2, 1, 0, 0, 0, 2, 2, 0, 2, 0, 2, 0, 0, 2, 1, 2, 2, 2, 2, 2, 0, 1,
       2, 0, 0, 2, 1, 1, 2, 2, 0, 2, 2, 0, 2, 2, 1, 0, 0, 0, 2, 0, 0, 2,
       2, 0, 2, 0, 0, 2, 1, 2, 1, 1, 2, 0, 0, 2, 2, 0, 2, 0, 2, 2, 1, 0,
       2, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 0, 2, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 2, 1, 2, 0, 0, 2, 0, 2, 0, 0, 2, 2, 0, 0, 2, 0, 1, 1, 0, 2,
       1, 2, 2, 0, 0, 0, 2, 1, 0, 1, 1, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 1,
       0, 2, 1, 1, 0, 2, 1, 1, 0, 2, 1, 0, 2, 0, 2,

In [23]:
print(utilities.pred_accuracy(all_labels_factored['scina'], merged_labels))
print(utilities.pred_accuracy(all_labels_factored['sctype'], merged_labels))
print(utilities.pred_accuracy(all_labels_factored['scsorter'], merged_labels))
#print(utilities.pred_accuracy(all_labels_factored['singler'], real_y))
#print(utilities.pred_accuracy(all_labels_factored['scpred'], real_y))


0.5404624342918396
0.7731214165687561
0.7601156234741211


In [25]:
max_pred = torch.tensor(encoded_labels).max(dim=1)[1]
utilities.pred_accuracy(max_pred, merged_labels)

/home/users/lewinsda/scSHARP/utilities.py:175: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return float((torch.tensor(preds) == torch.tensor(real)).type(torch.FloatTensor).mean().numpy())


0.8453757166862488

In [39]:
confident_labels = utilities.get_consensus_labels(encoded_labels, necessary_vote = 2)
confident_labels.shape

(692,)

In [40]:
train_nodes = np.where(confident_labels != -1)[0]
test_nodes = np.where(confident_labels == -1)[0]
len(test_nodes)

124

In [41]:
print(utilities.pred_accuracy(confident_labels[train_nodes], merged_labels[train_nodes]))

0.8785211443901062


In [33]:
# tool accuracy on test
print(utilities.pred_accuracy(np.array(all_labels_factored['scina'][test_nodes]), merged_labels[test_nodes]))
print(utilities.pred_accuracy(np.array(all_labels_factored['sctype'][test_nodes]), merged_labels[test_nodes]))
print(utilities.pred_accuracy(np.array(all_labels_factored['scsorter'][test_nodes]), merged_labels[test_nodes]))
#print(utilities.pred_accuracy(np.array(all_labels_factored['singler'][test_nodes]), real_y[test_nodes]))
#print(utilities.pred_accuracy(np.array(all_labels_factored['scpred'][test_nodes]), real_y[test_nodes]))
#max_pred = torch.tensor(encoded_labels).max(dim=1)[1]
print(utilities.pred_accuracy(max_pred[test_nodes], merged_labels[test_nodes]))

0.0
0.3790322542190552
0.2661290466785431
0.6935483813285828


In [43]:
all_labels_factored.iloc[test_nodes,:]

,scina,scsorter,sctype
10,-1,1,-1
19,-1,2,-1
22,-1,-1,-1
25,-1,-1,-1
29,-1,-1,0
...,...,...,...
672,-1,-1,-1
676,-1,1,0
680,-1,1,-1
684,-1,-1,-1


In [44]:
unique, counts = np.unique(merged_labels[test_nodes], return_counts=True)
counts

array([68, 47,  9])